<hr style="height: 1px;">
<i>This notebook was authored by the 8.S50x Course Team, Copyright 2022 MIT All Rights Reserved.</i>
<hr style="height: 1px;">
<br>

<h1>Lesson 17: Numerical ODE Simulations Part I</h1>


<a name='section_17_0'></a>
<hr style="height: 1px;">


## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L17.0 Overview</h2>


<h3>Navigation</h3>

<table style="width:100%">
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_17_1">L17.1 Simulating a Pendulum</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_17_1">L17.1 Exercises</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_17_2">L17.2 The Symplectic Approach</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_17_2">L17.2 Exercises</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_17_3">L17.3 The Leap-Frog Verlet Approach</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_17_3">L17.3 Exercises</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_17_4">L17.4 Runge-Kutta</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_17_4">L17.4 Exercises</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_17_5">L17.5 Modern High Quality Integrators</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_17_5">L17.5 Exercises</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_17_6">L17.6 Fitting the Full Thing</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_17_6">L17.6 Exercises</a></td>
    </tr>
</table>

In [ ]:
#>>>RUN: L17.0-runcell00

#Importing data:

!git init
!git remote add -f origin https://github.com/mitx-8s50/nb_LEARNER/
!git config core.sparseCheckout true
!echo 'data/L17' >> .git/info/sparse-checkout
!git pull origin main

In [ ]:
#>>>RUN: L17.0-runcell01

!pip install lmfit

In [ ]:
#>>>RUN: L17.0-runcell02

import imageio                        #https://imageio.readthedocs.io/en/stable/
from PIL import Image                 #https://pillow.readthedocs.io/en/stable/reference/Image.html

import numpy as np                    #https://numpy.org/doc/stable/
import torch                          #https://pytorch.org/docs/stable/torch.html
import torch.nn as nn                 #https://pytorch.org/docs/stable/nn.html
import matplotlib.pyplot as plt       #https://matplotlib.org/3.5.3/api/_as_gen/matplotlib.pyplot.html

from scipy.integrate import odeint    #https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.odeint.html
from scipy.optimize import minimize   #https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html
import csv                            #https://docs.python.org/3/library/csv.html
from matplotlib.patches import Circle #https://matplotlib.org/stable/api/_as_gen/matplotlib.patches.Circle.html
from scipy.integrate import solve_ivp #https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.solve_ivp.html

In [ ]:
#>>>RUN: L17.0-runcell03

#set plot resolution
%config InlineBackend.figure_format = 'retina'

#set default figure parameters
plt.rcParams['figure.figsize'] = (9,6)

medium_size = 12
large_size = 15

plt.rc('font', size=medium_size)          # default text sizes
plt.rc('xtick', labelsize=medium_size)    # xtick labels
plt.rc('ytick', labelsize=medium_size)    # ytick labels
plt.rc('legend', fontsize=medium_size)    # legend
plt.rc('axes', titlesize=large_size)      # axes title
plt.rc('axes', labelsize=large_size)      # x and y labels
plt.rc('figure', titlesize=large_size)    # figure title

<a name='section_17_1'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L17.1 Simulating a Pendulum</h2>  

| [Top](#section_17_0) | [Previous Section](#section_17_0) | [Exercises](#exercises_17_1) | [Next Section](#section_17_2) |


In [ ]:
#>>>RUN: L17.1-runcell01

#theta[0]=x
#theta[1]=v

def step(theta,dt,g,l):
    theta_tmp  = theta[0]
    theta_vtmp = theta[1] 
    theta[0] += theta_vtmp * dt
    theta[1] += (g/l)*(-1)*theta_tmp*dt

def store(theta,thetapos,time,timepos):
    thetapos = np.vstack((thetapos,theta))
    timepos  = np.vstack((timepos,time))
    return timepos,thetapos
    
def loop(istep, nsteps=10000,dt=0.01,g=9.8,l=1):
    timepos  = np.zeros(1)
    thetapos = np.zeros((1,2));
    time     = 0
    theta    = np.zeros(2)
    theta[0] = 0.1 #initial conditions
    theta[1] = 0.0
    timepos,thetapos=store(theta,thetapos,time,timepos)
    for timestep in range(nsteps):
        istep(theta,dt,g,l)
        time += dt
        timepos,thetapos=store(theta,thetapos,time,timepos)
    return timepos,thetapos

timepos,thetapos=loop(step,nsteps=1000)

plt.plot(timepos,thetapos[:,0],label="position")
plt.plot(timepos,thetapos[:,1],label="velocity")
plt.legend()
plt.xlabel("time(s)")
plt.ylabel("position")
plt.show()


In [ ]:
#>>>RUN: L17.1-runcell02

def energy(theta,g=9.8,l=1):
    kin=0.5*(l*theta[:,1])**2
    pot=-g*l*np.cos(theta[:,0])
    return kin+pot 

epos = energy(thetapos)

plt.plot(timepos,epos,label="energy")
plt.legend()
plt.xlabel("time(s)")
plt.ylabel("Energy(j)")
plt.show()

<a name='exercises_17_1'></a>     

| [Top](#section_17_0) | [Restart Section](#section_17_1) | [Next Section](#section_17_2) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Exercise 17.1.1</span>

Use the definitions for position and velocity shown below to explicitly compute the difference in energy for a step $\Delta t$. Specifically, calculate the difference in energy $\Delta \mathcal{H_{1}} = \mathcal{H_{1}}-\mathcal{H_{0}}$. Assume an initial angle of $\theta_0=0$ and a non-zero initial angular velocity $\dot{\theta}_0$.

$$
\theta_{n+1} = \theta_{n} + \Delta t \dot{\theta}_{n}\\
\dot{\theta}_{n+1} = \dot{\theta}_{n} - \Delta t \frac{g}{\ell} \theta_n \\
$$

Express your answer in terms of `m`, `g`, `l` for $\ell$, `Deltat` for $\Delta t$, and `thetadot_0` for $\dot{\theta}_0$. Make small angle approximations of any trig functions in your answer using the assumption that $\Delta t$ is very small.

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Exercise 17.1.2</span>

Show that the total change in energy varies with $\Delta t$ when stepping over the same total period of time. To do this, consider two different step sizes `dt1=0.001` and `dt2=0.01` and a total time of $10$. Create a plot of energy versus time for these choices of step size. Make sure that you step your simulation over the same total time: $n_{\rm steps} \times \Delta t$.

To check your code, calculate the magnitude of the fractional difference in energy between the last element of the `dt1=0.001` computation and the last energy using the `dt2=0.01` step size, i.e.,:

$$\mathrm{frac\_diff} = \left|\frac{\mathcal{H_{final}}(dt2)-\mathcal{H_{final}}(dt1)}{\mathcal{H_{final}}(dt1)}\right|$$.

Enter your answer as a number with precision `1e-3`.


In [ ]:
#>>>EXERCISE: L17.1.2

#keep time_total fixed
time_total = 10.

###################
#time-step 1:
dt1 = 0.001
nsteps1 = int(time_total/dt1)

timepos1,thetapos1=loop(step,nsteps=nsteps1,dt=dt1)
epos1 = energy(thetapos1)


###################
#time-step 2:

#YOUR CODE HERE

###################
#Make plots and compute
#fractional difference in energy

delta_H = #YOUR CODE HERE
print("frac_diff:", abs(delta_H/epos1[-1]))


#MAKE PLOTS

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Exercise 17.1.3</span>

Why is it important that energy is conserved in a numerical stepper?

A) Conserving energy ensures that the numerical solution accurately reflects the physical behavior of the system, providing realistic and meaningful results.

B) Energy conservation is often linked to the stability of the numerical method over long integration times, so methods that conserve energy are less likely to exhibit numerical artifacts or instability.

C) In systems with oscillatory motion, conserving energy helps maintain the correct frequency and amplitude of oscillations over time.

D) Deviations from energy conservation can lead to systematic errors in the solution.

E) Non-conservative methods may introduce unphysical damping or amplification, leading to incorrect results and potentially misleading interpretations of the system's behavior.

<a name='section_17_2'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L17.2 The Symplectic Approach</h2>  

| [Top](#section_17_0) | [Previous Section](#section_17_1) | [Exercises](#exercises_17_2) | [Next Section](#section_17_3) |


In [ ]:
#>>>RUN: L17.2-runcell01

def step(theta,dt,g,l):
    theta_tmp  = theta[0]
    theta_vtmp = theta[1] 
    theta[0] += theta_vtmp * dt + 0.5*(g/l)*(-1)*np.sin(theta_tmp)*dt**2 #comment out higher-order term if desired
    theta[1] += (g/l)*(-1)*np.sin(theta_tmp)*dt
    #theta[1] += (g/l)*(-1)*theta_tmp*dt

    
def energy(theta,g=9.8,l=1):
    kin=0.5*l*l*theta[:,1]**2
    pot=-g*l*np.cos(theta[:,0])
    return kin+pot 

timepos,thetapos=loop(step,nsteps=1000,dt=0.01)

plt.plot(timepos,thetapos[:,0],label="position")
plt.plot(timepos,thetapos[:,1],label="velocity")
plt.legend()
plt.xlabel("time(s)")
plt.ylabel("position or velocity")
plt.show()

timepos,thetapos=loop(step,nsteps=1000,dt=0.01)
epos = energy(thetapos)

plt.plot(timepos[1:-1],epos[1:-1],label="energy")
plt.legend()
plt.xlabel("time(s)")
plt.ylabel("energy")
plt.show()


<a name='exercises_17_2'></a>     

| [Top](#section_17_0) | [Restart Section](#section_17_2) | [Next Section](#section_17_3) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Exercise 17.2.1</span>

Compare the energy conservation for a quadratic approximation of the Euler method vs the linear approximation, over a period of 10s. Make any relevant plots to help your analysis.

To check your result, calculate the magnitude of the fractional difference in energy between the last elements of your computation, relative to the energy computed using the quadratic approximation, i.e.,:

$$\mathrm{frac\_diff} = \left|\frac{\mathcal{H_{linear}}(10s)-\mathcal{H_{quadratic}}(10s)}{\mathcal{H_{quadratic}}(10s)}\right|$$.

Enter your answer as a number with precision `1e-3`.

In [ ]:
#>>>EXERCISE: L17.2.1

#Functions
##########

def stepl(theta,dt,g,l):
    #linear stepper
    theta_tmp  = theta[0]
    theta_vtmp = theta[1] 
    theta[0] += theta_vtmp * dt 
    theta[1] += (g/l)*(-1)*np.sin(theta_tmp)*dt
    
def stepq(theta,dt,g,l):
    #quadratic stepper
    #YOUR CODE HERE


#run steps
##########

timeposl,thetaposl=loop(stepl,nsteps=1000,dt=0.01)
timeposq,thetaposq=loop(stepq,nsteps=1000,dt=0.01)
eposl = energy(thetaposl)
eposq = energy(thetaposq)


###################
#Make plots and compute
#fractional difference in energy

delta_H = #YOUR CODE HERE
print("frac_diff:", abs(delta_H/eposq[-1]))

<a name='section_17_3'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L17.3 The Leap-Frog Verlet Approach</h2>  

| [Top](#section_17_0) | [Previous Section](#section_17_2) | [Exercises](#exercises_17_3) | [Next Section](#section_17_4) |


In [ ]:
#>>>RUN: L17.3-runcell01


def altstep(theta,dt,g,l):
    theta[0] += theta[1] * dt 
    theta[1] += (g/l)*(-1)*np.sin(theta[0])*dt
    
def loop(istep,nsteps=1000,dt=0.01,g=9.8,l=1):
    timepos  = np.zeros(1)
    thetapos = np.zeros((1,2));
    time     = 0
    theta    = np.zeros(2)
    theta[0] = 0.1 #initial conditions
    theta[1] = 0.0
    #theta[1] += (g/l)*(-1)*theta[0]*dt*0.5 #half step
    timepos,thetapos=store(theta,thetapos,time,timepos)
    for timestep in range(nsteps):
        istep(theta,dt,g,l)
        time += dt
        timepos,thetapos=store(theta,thetapos,time,timepos)
    return timepos,thetapos


timepos,thetapos=loop(altstep,nsteps=1000,dt=0.01)
plt.plot(timepos,thetapos[:,0],label="position")
plt.plot(timepos,thetapos[:,1],label="velocity")
plt.legend()
plt.xlabel("time(s)")
plt.ylabel("position or velocity")
plt.show()

epos = energy(thetapos)

plt.plot(timepos[1:-1],epos[1:-1],label="energy")
plt.legend()
plt.xlabel("time(s)")
plt.ylabel("energy")
plt.show()


In [ ]:
#>>>RUN: L17.3-runcell02

def lfstep(theta,dt,g,l):
    theta[1] += 0.5*(g/l)*(-1)*np.sin(theta[0])*dt
    theta[0] += theta[1] * dt 
    #theta[1] += (g/l)*(-1)*theta[0]*dt
    theta[1] += 0.5*(g/l)*(-1)*np.sin(theta[0])*dt
    
def loop(istep,nsteps=1000,dt=0.01,g=9.8,l=1):
    timepos  = np.zeros(1)
    thetapos = np.zeros((1,2));
    time     = 0
    theta    = np.zeros(2)
    theta[0] = 0.1 #initial conditions
    theta[1] = 0.0
    #theta[1] += (g/l)*(-1)*theta[0]*dt*0.5 #half step
    timepos,thetapos=store(theta,thetapos,time,timepos)
    for timestep in range(nsteps):
        istep(theta,dt,g,l)
        time += dt
        timepos,thetapos=store(theta,thetapos,time,timepos)
    return timepos,thetapos


timepos,thetapos=loop(lfstep,nsteps=1000,dt=0.01)
plt.plot(timepos,thetapos[:,0],label="position")
plt.plot(timepos,thetapos[:,1],label="velocity")
plt.legend()
plt.xlabel("time(s)")
plt.ylabel("position or velocity")
plt.show()

epos = energy(thetapos)
print("E0:",epos[1],"E1:",epos[-1])

plt.plot(timepos[1:-1],epos[1:-1],label="energy")
plt.legend()
plt.xlabel("time(s)")
plt.ylabel("energy")
plt.show()


<a name='exercises_17_3'></a>     

| [Top](#section_17_0) | [Restart Section](#section_17_3) | [Next Section](#section_17_4) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Exercise 17.3.1</span>

Take a closer look at the position, velocity, and energy distributions above. How does the frequency of the energy variation compare to the natural oscillation frequency of the system? Where does this variation come from?

The frequency of energy oscillation is

A) equal to the natural frequency of oscillation of the system.\
B) half the natural frequency of oscillation of the system.\
C) double the natural frequency of oscillation of the system.


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Exercise 17.3.2</span>

How is energy conserved in the Leap-Frog Verlet procedure? Choose the most appropriate option:

A) The procedure adjusts energy explicitly at each time step to ensure conservation.

B) Energy conservation is achieved through random position updates.

C) Energy conservation results from staggered position and velocity updates.

D) The procedure continuously corrects velocity to maintain consistent energy conservation.



<a name='section_17_4'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L17.4 Runge-Kutta</h2>  

| [Top](#section_17_0) | [Previous Section](#section_17_3) | [Exercises](#exercises_17_4) | [Next Section](#section_17_5) |


In [ ]:
#>>>RUN: L17.4-runcell01

def thetaa(itheta,g,l):
    return -(g/l)*np.sin(itheta)   

def thetav(ithetad):
    return ithetad# - (g/l)*np.sin(itheta)*idt

def rkstep(theta,dt,g,l):
    thetan    = theta[0]
    thetavn   = theta[1]
    thetaan   = thetaa(thetan,g,l)
    #theta[0] += dt*0.5*(thetavn+thetav(thetavn,dt*0.5,thetan,g,l))#current + extrapolated 
    #theta[1] += dt*0.5*(thetaa(theta[0],0,0,g,l) + thetaa(theta[0],dt,thetavn,g,l))
    theta[0] += 0.5*dt*(thetavn + thetav(thetavn+dt*thetaa(thetan,g,l)    ))
    theta[1] += 0.5*dt*(thetaan + thetaa(thetan +dt*thetav(thetavn   ),g,l))

def loop(nsteps=100000,dt=0.01,g=9.8,l=1):
    timepos  = np.zeros(1)
    thetapos = np.zeros((1,2));
    time     = 0
    theta    = np.zeros(2)
    theta[0] = 0.1 #initial conditions
    theta[1] = 0.0
    theta[1] += dt*0.5*(thetaa(theta[0],g,l))
    timepos,thetapos=store(theta,thetapos,time,timepos)
    for timestep in range(nsteps):
        rkstep(theta,dt,g,l)
        time += dt
        timepos,thetapos=store(theta,thetapos,time,timepos)
    return timepos,thetapos


timepos,thetapos=loop(nsteps=10000,dt=0.01)

plt.plot(timepos,thetapos[:,0],label="position")
plt.plot(timepos,thetapos[:,1],label="velocity")
plt.legend()
plt.xlabel("time(s)")
plt.ylabel("position or velocity")
plt.show()

epos = energy(thetapos)

plt.plot(timepos[1:-1],epos[1:-1],label="energy")
plt.legend()
plt.xlabel("time(s)")
plt.ylabel("energy")
plt.show()



In [ ]:
#>>>RUN: L17.4-runcell02


def rkstep(theta,dt,g,l):
    thetan     = theta[0]
    thetavn    = theta[1]
    thetaan    = thetaa(thetan,g,l)    
    #half step
    thetavn05  = thetav(thetavn  +0.5*dt*thetaan) 
    thetaan05  = thetaa(thetan   +0.5*dt*thetavn,g,l) 
    #half step based on previous half
    thetavn05d = thetav(thetavn  +0.5*dt*thetaan05)    
    thetaan05d = thetaa(thetan   +0.5*dt*thetavn05,g,l)
    #full step based on previous half
    thetavn10  = thetav(thetavn  +1.0*dt*thetaan05d)
    thetaan10  = thetaa(thetan   +1.0*dt*thetavn05d,g,l)
    #runge-Kutta did the math for us
    theta[0] += (1./6.)*dt*(thetavn + 2*thetavn05  + 2*thetavn05d + thetavn10)
    theta[1] += (1./6.)*dt*(thetaan + 2*thetaan05  + 2*thetaan05d + thetaan10)

timepos,thetapos=loop(nsteps=10000,dt=0.01)

plt.plot(timepos,thetapos[:,0],label="position")
plt.plot(timepos,thetapos[:,1],label="velocity")
plt.legend()
plt.xlabel("time(s)")
plt.ylabel("position or velocity")
plt.show()

epos = energy(thetapos)

plt.plot(timepos[1:-1],epos[1:-1],label="energy")
plt.legend()
plt.xlabel("time(s)")
plt.ylabel("energy")
plt.show()

print("E-start:",epos[1],"E-end",epos[-1])

<a name='exercises_17_4'></a>     

| [Top](#section_17_0) | [Restart Section](#section_17_4) | [Next Section](#section_17_5) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Exercise 17.4.1</span>

Let's now look at the pendulum in the non-linear regime, namely larger angles where the approximation $\sin(\theta)\approx\theta$ no longer holds. Compare the Runge-Kutta method to the leap-frog method for an initial angle $\theta_0=0.9$ radians. How does the conservation of energy for each method compare, using a fixed time-step of `dt=0.01`?

To answer this question, complete the code below to create your desired plots (the functions `rkloop` and `lfloop` are already defined for you, corresponding to the Runge-Kutta and leap-frog methods, respectively).

Select ALL that apply:


A) Both the RK and LF methods show comparable changes in energy over time.\
B) The RK method shows larger variations in energy over time than the LF method.\
C) The RK method shows smaller variations in energy over time than the LF method.\
D) The average energy using the RK method appears to decrease over time, whereas that for the LF method does not.\
E) The average energy using the LF method appears to decrease over time, whereas that for the RK method does not.



In [ ]:
#>>>EXERCISE: L17.4.1

thetainit=0.9 #units of radians
def rkloop(total_time=10,dt=0.01,g=9.8,l=1):
    nsteps = int(total_time/dt)
    timepos  = np.zeros(1)
    thetapos = np.zeros((1,2));
    time     = 0
    theta    = np.zeros(2)
    theta[0] = thetainit #initial conditions
    theta[1] = 0.0
    theta[1] += dt*0.5*(thetaa(theta[0],g,l))
    timepos,thetapos=store(theta,thetapos,time,timepos)
    for timestep in range(nsteps):
        rkstep(theta,dt,g,l)
        time += dt
        timepos,thetapos=store(theta,thetapos,time,timepos)
    return timepos,thetapos

def lfloop(total_time=10,dt=0.01,g=9.8,l=1):
    nsteps = int(total_time/dt)
    timepos  = np.zeros(1)
    thetapos = np.zeros((1,2));
    time     = 0
    theta    = np.zeros(2)
    theta[0] = thetainit #initial conditions
    theta[1] = 0.0
    theta[1] += dt*0.5*(thetaa(theta[0],g,l))
    timepos,thetapos=store(theta,thetapos,time,timepos)
    for timestep in range(nsteps):
        lfstep(theta,dt,g,l)
        time += dt
        timepos,thetapos=store(theta,thetapos,time,timepos)
    return timepos,thetapos



#PLOT COMPARISONS HERE


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Exercise 17.4.2</span>

Now change the timestep to `dt=0.2` in the code that you wrote for the previous problem. How do the results compare? Select ALL that apply:


A) Both the RK and LF methods show comparable changes in energy over time.\
B) The RK method shows larger variations in energy over time than the LF method.\
C) The RK method shows smaller variations in energy over time than the LF method.\
D) The average energy using the RK method appears to decrease over time, whereas that for the LK method does not.\
E) The average energy using the LF method appears to decrease over time, whereas that for the RK method does not.


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Exercise 17.4.3</span>

How do the Runge-Kutta Algorithm and Leap-Frog Verlet procedures differ? Select ALL that apply:

A) Runge-Kutta methods generally achieve a higher order of accuracy compared to the Leap-Frog Verlet procedure.

B) The Leap-Frog Verlet procedure is a symplectic integrator, which conserves energy, while Runge-Kutta methods are not inherently symplectic.

C) The Leap-Frog Verlet procedure updates position and velocity at different points in time, leading to a staggered structure, whereas Runge-Kutta methods use weighted averages of multiple function evaluations at each time step.

<a name='section_17_5'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L17.5 Modern High Quality Integrators</h2>  

| [Top](#section_17_0) | [Previous Section](#section_17_4) | [Exercises](#exercises_17_5) | [Next Section](#section_17_6) |


In [ ]:
#>>>RUN: L17.5-runcell01

def df(t, thetavec, g, l):
    theta, thetadot = thetavec
    return [thetadot, -(g/l) * np.sin(theta)]

g=9.8
l=1
thetainit=2.0
solution = solve_ivp(df, [0., 10000*0.01], [thetainit, 0.], max_step = 0.01, args=(g,l))


plt.plot(solution.t, solution.y[0],label="int-position")
plt.plot(solution.t, solution.y[1],label="int-velocity")
plt.xlabel("time(s)")
plt.ylabel("position or velocity")
plt.legend()
plt.show()

#redefining energy
def energy(theta,g=9.8,l=1):
    kin=0.5*l*l*theta[:,1]**2
    pot=-g*l*np.cos(theta[:,0])
    return kin+pot 

energypos=energy(solution.y.T)
#e = 0.5 * m * solution.y[1]**2 + 0.5 * k * solution.y[0]**2
plt.plot(solution.t, energypos)
plt.xlabel("time(s)")
plt.ylabel("energy")
plt.show()

print("E-start:",energypos[1],"E-end:",energypos[-1])


In [ ]:
#>>>RUN: L17.5-runcell02

def df(t, thetavec, g, l, mup, fp, omega):
    theta, thetadot = thetavec
    return [thetadot, -(g/l) * np.sin(theta) - mup*thetadot + fp*np.cos(omega*t) ]

g=9.8
l=1
thetainit=2.0
mup=0.1
fp=0.1
# The following line sets the driving frequency as a fraction of the natural
# frequency of the system (g/l).
omega=np.sqrt(0.85*g/l)
solution = solve_ivp(df, [0., 10000*0.01], [thetainit, 0.], max_step = 0.01, args=(g,l,mup,fp,omega))

plt.plot(solution.t, solution.y[0],label="int-position")
plt.plot(solution.t, solution.y[1],label="int-velocity")
plt.xlabel("time(s)")
plt.ylabel("position or velocity")
plt.legend()
plt.show()

energypos=energy(solution.y.T)
#e = 0.5 * m * solution.y[1]**2 + 0.5 * k * solution.y[0]**2
plt.plot(solution.t, energypos)
plt.xlabel("time(s)")
plt.ylabel("energy")
plt.show()

print("E-start:",energypos[1],"E-end:",energypos[-1])


In [ ]:
#>>>RUN: L17.5-runcell03


thetainit=2.0
mu=0.1
fp=0.1
l=1

omega=np.sqrt(0.99*g/l)
solution1 = solve_ivp(df, [0., 10000*0.01], [thetainit, 0.], max_step = 0.01, args=(g,l,mu,fp,omega))

omega=np.sqrt(1.02*g/l)
solution2 = solve_ivp(df, [0., 10000*0.01], [thetainit, 0.], max_step = 0.01, args=(g,l,mu,fp,omega))

plt.plot(solution.y[0], solution.y[1],label="0.85")
plt.plot(solution1.y[0], solution1.y[1],label="0.99")
plt.plot(solution2.y[0], solution2.y[1],label="1.02")
plt.xlabel("position")
plt.ylabel("velocity")
plt.legend()
plt.show()


In [ ]:
#>>>RUN: L17.5-runcell04

for scale in np.arange(0.8,1.20,0.01):
    omega=np.sqrt(scale*g/l)
    solution = solve_ivp(df, [0., 10000*0.01], [thetainit, 0.], max_step = 0.1, args=(g,l,mu,fp,omega))
    plt.plot(solution.y[0], solution.y[1],linewidth=0.1)


plt.xlabel("position")
plt.ylabel("velocity")
plt.show()

<a name='exercises_17_5'></a>     

| [Top](#section_17_0) | [Restart Section](#section_17_5) | [Next Section](#section_17_6) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Exercise 17.5.1</span>

In the limit of small oscillations, there is analytic solution for a damped-driven harmonic oscillator, and it has two terms, corresponding to steady-state and transient solutions, respectively. The point of this problem will be to see how well this exact solution compares to the numerical solution in the limit of small initial angle and small driving force.

$$
\theta_{\mathrm{total}} = \theta_{\mathrm{transient}} + \theta_{\mathrm{steady-state}}
$$

Firstly, the <b>steady-state solution</b> is largely governed by the driving frequency $\Omega$, and it is given by:

$$
\theta_{\mathrm{steady-state}} = A \cos(\Omega t - \phi)
$$

where the amplitude and phase are defined as:

$$
A = \frac{f'}{\sqrt{(\omega_0^2 - \Omega^2)^2 + (\mu' \Omega)^2}}
$$

$$
\phi = \arctan\left(\frac{\mu' \Omega}{\omega_0^2 - \Omega^2}\right)
$$

with the natural frequency $\omega_0 = \sqrt{\frac{g}{\ell}}$.


Next, the <b>transient solution</b> decays as a function of the damping, as follows (for the underdamped regime):

$$
\theta_{\mathrm{transient}} = e^{-\frac{\mu'}{2}t}\left(C\cos(\omega_t t) + D\cos(\omega_t t)\right)
$$

where the frequency of transient oscillations is:

$$
\omega_t = \sqrt{\omega_0^2 - \left(\frac{\mu'}{2}\right)^2}
$$


and $C$ and $D$ are determined by initial conditions. For the generalized case of initial angular position $\theta(t=0)=\theta_0$ and initial angular velocity $\dot{\theta}(t=0)=\dot{\theta}_0$:

$$
C = \theta_0 - A\cos(\Omega t)
$$

$$
D = \frac{1}{\omega_t}\left(\dot{\theta}_0 + \frac{\mu'}{2}C - \Omega A\sin(\Omega t) \right)
$$


Plot this analytic solution alongside the numerical solution (to make it simpler for viewing, you could look at the last `1000` pts of the simulation). What is the fractional difference in amplitude between the two solutions, relative to the theoretical amplitude $A$, evaluated at the very last time step (i.e., the last element of the array)? In other words, compute the following and report your answer with precision `1e-4`:

$$
\mathrm{frac\_diff} = \left|\dfrac{\theta_{numerical} - \theta_{analytic}}{A}\right|
$$


In [ ]:
#>>>EXERCISE: L17.5.1

def df(t, thetavec, g, l, mup, fp, omega):
    theta, thetadot = thetavec
    return [thetadot, -(g/l) * np.sin(theta) - mup*thetadot + fp*np.cos(omega*t)]

# Constants
g = 9.8
l = 1.
thetainit = 0.1
thetadotinit = 0.
mup = 0.1
fp = 0.01
omega = np.sqrt(0.85 * g / l)

# Solve the differential equation numerically
solution = solve_ivp(df, [0., 10000*0.01], [thetainit, thetadotinit], max_step=0.01, args=(g, l, mup, fp, omega))

# Analytic solution
#YOUR CODE HERE

# Plot the numerical and analytic solutions
#YOUR CODE HERE

# Print the fractional difference
#YOUR CODE HERE

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Exercise 17.5.2</span>

The analytic solution described above is only valid in the limit of small oscillations. Test the accuracy of the analytic solution by varying the magnitude of the driving force $f'$. Approximately what is the minimal value of $f'$ that leads to a $1\%$ difference in the angular amplitudes of the two solutions, relative to the theoretical amplitude $A$, evaluated at the very last time step (keeping all other parameters fixed). Report your answer with precision `1e-2`.

Hint: You can view the solution to the previous problem if you need help formulating an answer.

<a name='section_17_6'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L17.6 Fitting the Full Thing</h2>  

| [Top](#section_17_0) | [Previous Section](#section_17_5) | [Exercises](#exercises_17_6) |

In [ ]:
#>>>RUN: L17.6-runcell01

#Jade & Kiran
def load(iFile='data/L17/jade-pendulum.csv'):
    times=np.array([])
    amps =np.array([])
    with open(iFile, newline='') as csvfile:
        line = csv.reader(csvfile, delimiter=' ', quotechar='|')
        for row in line:
            pT=float(row[0])
            pA=float(row[1])
            times=np.append(pT,times)
            amps =np.append(pA,amps)
    #Transform
    amps  = (amps-amps.mean())/(2.*np.max(amps))
    return times,amps

tk,ak=load('data/L17/Kiran.csv')
tj,aj=load()


plt.plot(tj,aj,'.',label='Jade')
plt.plot(tk,ak,'.',label='Kiran')
plt.xlabel('time(s)')
plt.ylabel("Angular Amplitude")
plt.legend()
plt.show()

plt.plot(tj[0:1000],aj[0:1000],'.',label='Jade')
plt.plot(tk[0:1000],ak[0:1000],'.',label='Kiran')
plt.xlabel('time(s)')
plt.ylabel("Angular Amplitude")
plt.legend()
plt.show()

In [ ]:
#>>>RUN: L17.6-runcell02

plt.plot(tj[0:10],aj[0:10],'.',label='Jade')
plt.xlabel('time(s)')
plt.ylabel("Angular Amplitude")
plt.show()

plt.plot(tj[5300:5310],aj[5300:5310],'.',label='Jade')
plt.xlabel('time(s)')
plt.ylabel("Angular Amplitude")
plt.show()

plt.plot(tk[0:10],ak[0:10],'.',label='Kiran')
plt.xlabel('time(s)')
plt.ylabel("Angular Amplitude")
plt.show()



In [ ]:
#>>>RUN: L17.6-runcell03

from lmfit import Model, Parameter, report_fit, fit_report
from lmfit.models import LinearModel

model = LinearModel()
pars = model.make_params(intercept=0, slope=0)
result=model.fit(x=tk[0:10],data=ak[0:10],pars=pars,weights=1./(np.ones(10)*2e-4))
print(result.fit_report())
result.plot_fit()

In [ ]:
#>>>RUN: L17.6-runcell04

time=np.flip(tk)[0:1000]
amp= np.flip(ak)[0:1000]
weights=1.0/(np.ones(len(amp))*2*1e-4)
lk = 10.7886
lj = 4.151
l=lk

def ffull(y, t, g, l, mu): #This is for ODE
    theta, thetadot = y
    return [thetadot, - (g / l) * np.sin(theta)-mu*thetadot]


def func(t,x0,x1,x2,x3,x4,x5):
    #solution = solve_ivp(ffull,[time[0],time[-1]], [x0, x1], max_step = 0.1, args=(x2,l,x3),t_eval=t)
    #solution = x4*solution.y[0]+x5#np.sin(solution[:,0])+x[5]
    solution = odeint(ffull, [x0,x1,], t, args=(x2,l,x3))
    solution = x4*solution[:,0]+x5
    return solution

plt.plot(time,amp,'.',label='Kiran')
plt.xlabel('time(s)')
plt.ylabel("Angular Amplitude")
plt.show()

xtest = np.array([-0.01,0.01,9.8,0.0,15,0.01])
model = Model(func, independent_vars=['t'])
params = model.make_params()
params['x0'].min=np.min(amp)*2
params['x0'].max=np.max(amp)*2
params['x1'].min=np.min(amp)*10
params['x1'].max=np.max(amp)*10
params['x2'].min=9.5
params['x2'].max=10.0
params['x3'].min=-0.1
params['x3'].max=0.1
params['x4'].min=1
params['x4'].max=100
#params['x4'].vary=False
params['x5'].min=-0.1
params['x5'].max=0.1
#params['x6'].min=-0.5
#params['x6'].max=0.5
#params['x6'].vary=False
result = model.fit(amp, t=time,params=params, weights=weights,
                  x0=xtest[0],x1=xtest[1],x2=xtest[2],x3=xtest[3],x4=xtest[4],x5=xtest[5])
print(result.fit_report())
result.plot_fit()
plt.show()
result.plot_residuals()
plt.show()



In [ ]:
#>>>RUN: L17.6-runcell05

time=np.flip(tk)[0:5000][::50]
amp= np.flip(ak)[0:5000][::50]
weights=1/(np.ones(len(amp))*8*1e-4) #inflate uncertainty
lk = 10.7886
lj = 4.151
l=lk
result = model.fit(amp, t=time,params=params, weights=weights,
                  x0=xtest[0],x1=xtest[1],x2=xtest[2],x3=xtest[3],x4=xtest[4],x5=xtest[5])
print(result.fit_report())
result.plot_fit()
plt.show()
result.plot_residuals()
plt.show()




In [ ]:
#>>>RUN: L17.6-runcell06

time=np.flip(tk)[5000:10000][::20]
amp= np.flip(ak)[5000:10000][::20]
weights=1/(np.ones(len(amp))*8*1e-4) #inflate uncertainty
lk = 10.7886
lj = 4.151
l=lk
xtest[4]=20
params['x4'].vary=False
result = model.fit(amp, t=time,params=params, weights=weights,
                  x0=xtest[0],x1=xtest[1],x2=xtest[2],x3=xtest[3],x4=xtest[4],x5=xtest[5])
print(result.fit_report())
result.plot_fit()
plt.show()
result.plot_residuals()
plt.show()

time=np.flip(tj)[5000:10000][::20]
amp= np.flip(aj)[5000:10000][::20]
weights=1/(np.ones(len(amp))*8*1e-4) #inflate uncertainty
lk = 10.7886
lj = 4.151
l=lj
xtest[4]=20
params['x4'].vary=False
result = model.fit(amp, t=time,params=params, weights=weights,
                  x0=xtest[0],x1=xtest[1],x2=xtest[2],x3=xtest[3],x4=xtest[4],x5=xtest[5])

print(result.fit_report())
result.plot_fit()
plt.show()
result.plot_residuals()
plt.show()




In [ ]:
#>>>RUN: L17.6-runcell07

def ffull(y, t, g, l, mu, drag): #This is for ODE
    theta, thetadot = y
    return [thetadot, - (g / l) * np.sin(theta)-mu*thetadot-drag*thetadot**2 ]


def func(t,x0,x1,x2,x3,x4,x5,x6):
    #solution = solve_ivp(ffull,[time[0],time[-1]], [x0, x1], max_step = 0.1, args=(x2,l,x3),t_eval=t)
    #solution = x4*solution.y[0]+x5#np.sin(solution[:,0])+x[5]
    solution = odeint(ffull, [x0,x1,], t, args=(x2,l,x3,x6))
    solution = (x4*solution[:,0]+x5)
    return solution


xtest = np.array([-0.01,0.01,9.8,0.0,20,0.01,0.0,0])
model = Model(func, independent_vars=['t'])
params = model.make_params()
params['x0']
params['x0'].min=np.min(amp)
params['x0'].max=np.max(amp)
params['x1'].min=np.min(amp)*10
params['x1'].max=np.max(amp)*10
params['x2'].min=9.5
params['x2'].max=10.0
params['x3'].min=-0.1
params['x3'].max=0.1
params['x4'].min=1
params['x4'].max=100
params['x4'].vary=False
params['x5'].min=-0.1
params['x5'].max=0.1
params['x6'].min=-10.5
params['x6'].max=10.5
#params['x7'].min=-0.1
#params['x7'].max=0.1
#params['x6'].vary=False



In [ ]:
#>>>RUN: L17.6-runcell08

time=np.flip(tk)[0:10000][::10]
amp= np.flip(ak)[0:10000][::10]
weights=1/(np.ones(len(amp))*8*1e-4) #inflate uncertainty
lk = 10.7886
lj = 4.151
l=lk
xtest[4]=20
params['x4'].vary=False
result = model.fit(amp, t=time,params=params, weights=weights,
                  x0=xtest[0],x1=xtest[1],x2=xtest[2],x3=xtest[3],x4=xtest[4],x5=xtest[5],x6=xtest[6])#,x7=xtest[7])
print(result.fit_report())
result.plot_fit()
plt.show()
result.plot_residuals()
plt.show()

time=np.flip(tj)[5000:10000][::10]
amp= np.flip(aj)[5000:10000][::10]
weights=1/(np.ones(len(amp))*8*1e-4) #inflate uncertainty
lk = 10.7886
lj = 4.151
l=lj
xtest[4]=20
params['x4'].vary=False
result = model.fit(amp, t=time,params=params, weights=weights,
                  x0=xtest[0],x1=xtest[1],x2=xtest[2],x3=xtest[3],x4=xtest[4],x5=xtest[5],x6=xtest[6])#,x7=xtest[7])

print(result.fit_report())
result.plot_fit()
plt.show()
result.plot_residuals()
plt.show()

def gtrue(phi):
    return 9.80616-0.025925*np.cos(2*phi)+0.000069*np.cos(phi)**2

gtrue(42./180.*np.pi)


<a name='exercises_17_6'></a>     

| [Top](#section_17_0) | [Restart Section](#section_17_6) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Exercise 17.6.1</span>

Repeat the fit to both datasets using the exact solution for a damped harmonic oscillator, defined as `dampfunc` in the code below. The analytic form of the solution is:

$$
\theta(t) = Be^{-\frac{\mu'}{2}t}\cos(\omega_t t + \delta)
$$

where $B$ and $\delta$ are parameters set based on intial conditionas, and the frequency of transient oscillations is given by:

$$
\omega_t = \sqrt{\omega_0^2 - \left(\frac{\mu'}{2}\right)^2}
$$

As with the model used in this section, you should add an additional offset term, such that the fit model resembles:

$$
\theta(t) = Be^{-\frac{\mu'}{2}t}\cos(\omega_t t + \delta) + \Delta
$$


What values do you obtain for $g$ for both data sets? Report your result as a list of two numbers with precision `1e-4`.

In [ ]:
#>>>EXERCISE: L17.6.1

def dampfunc(t,x0,x1,x2,x3,x4):
    return #YOUR CODE HERE


dampmodel = Model(dampfunc, independent_vars=['t'])
params = dampmodel.make_params()

xtest = np.array([]) #FILL IN ARRAY
#SET MIN/MAX ON PARAM VALS HERE

time=np.flip(tk)[0:10000][::10]
amp= np.flip(ak)[0:10000][::10]
weights=1/(np.ones(len(amp))*8*1e-4) #inflate uncertainty
lk = 10.7886
lj = 4.151
l=lk
result = dampmodel.fit(amp, t=time,params=params, weights=weights,
                      x0=xtest[0],x1=xtest[1],x2=xtest[2],x3=xtest[3],x4=xtest[5])
print(result.fit_report())
result.plot_fit()
plt.show()
result.plot_residuals()
plt.show()

time=np.flip(tj)[0:10000][::10]
amp= np.flip(aj)[0:10000][::10]
weights=1/(np.ones(len(amp))*8*1e-4) #inflate uncertainty
lk = 10.7886
lj = 4.151
l=lj
result = dampmodel.fit(amp, t=time,params=params, weights=weights,
                      x0=xtest[0],x1=xtest[1],x2=xtest[2],x3=xtest[3],x4=xtest[5])

print(result.fit_report())
result.plot_fit()
plt.show()
result.plot_residuals()
plt.show()